In [1]:
import numpy as np
from aeon.datasets import load_classification
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
from models import Models
from morph import Morph
from sklearn.preprocessing import LabelEncoder

In [8]:
medical_datasets = ['NerveDamage', 'MedicalImages']
ECG_datasets = ['AtrialFibrillation', 'CardiacArrhythmia', 'CinCECGTorso',
                'ECG200', 'ECG5000', 'ECGFiveDays', 'NonInvasiveFetalECGThorax1',
                'NonInvasiveFetalECGThorax2', 'StandWalkJump', 'TwoLeadECG']


X, y = load_classification("MedicalImages")
print("Shape of X = ", X.shape)

le = LabelEncoder()
y = le.fit_transform(y)
print(f'Classes: {np.unique(y)}')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Shape of X =  (1141, 1, 99)
Classes: [0 1 2 3 4 5 6 7 8 9]


In [ ]:
# Train Model
lstm = Models('lstm', X_train, y_train)
lstm.train_lstm()
catch = Models('catch22', X_train, y_train)
catch.train_catch22()
rocket = Models('rocket', X_train, y_train)
rocket.train_rocket()

In [10]:
results = {}
for c in np.unique(y):
    res = {}

    mor = Morph(X_test, y_test, c)
    mor.get_DTWGlobalBorderline(perc_samples=0.1)

    res['lsmt'] = mor.Binay_MorphingCalculater(lstm)
    res['catch'] = mor.Binay_MorphingCalculater(catch)
    res['rocket'] = mor.Binay_MorphingCalculater(rocket)

    results[c] = res
    
with open('results/MedicalImages.pkl', 'wb') as f:
    pickle.dump(res, f)


c:\Users\anton\anaconda3\envs\morph\lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


KeyboardInterrupt: 

In [ ]:
with open('results/MedicalImages.pkl', 'rb') as f:
    results = pickle.load(f)

In [ ]:
columns = ['class', 'model', 'mean', 'std']

df = []

for c in results.keys():
    for model in results[c].keys():
        data = results[c][model][3]
        line = [c, model, data['mean'], data['std']]
        df.append(line)

# save to csv
df = pd.DataFrame(df, columns=columns)
df.to_csv('results/medicalimages.csv', index=False)

{'mean': 0.44828451882845183, 'std': 0.23603946777203888}